In [ ]:
# Data
import numpy
import pandas as pd

import os

# Visualizaciones
import matplotlib.pyplot as plt

# Redes Neuronales
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

from keras.layers import Input, Dense, RandomFlip, RandomContrast, BatchNormalization, RandomTranslation
from keras.layers import Conv2D, Activation, GlobalMaxPooling2D, MaxPooling2D, Rescaling, Flatten, Dropout
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
REPO_NAME = "Multi-Parameter_Wireless_Network"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/MiguelMussi/{REPO_NAME}.git
  os.chdir(REPO_NAME)

Establecer GPU por defecto en caso de estar disponible.

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

In [ ]:
df = pd.read_txt("data.txt")

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=123)

In [ ]:
df_train.shape, df_test.shape

## Filas Duplicadas

In [ ]:
duplicadas = df_train.duplicated().any()
print("Hay filas duplicadas.") if duplicadas.any() else print("No hay filas duplicadas.")

In [ ]:
num_duplicados = df_train.duplicated().sum()
num_duplicados

In [ ]:
filas_duplicadas = df_train[df_train.duplicated()]
filas_duplicadas

In [ ]:
def eliminar_registros_duplicados(df):
    """
    Elimina los registros duplicados de un DataFrame.

    Parámetros:
    - df: DataFrame: El DataFrame que se va a limpiar de registros duplicados.

    Retorna:
    - df_clean: DataFrame: El DataFrame sin registros duplicados.
    """
    df_clean = df.drop_duplicates()
    return df_clean

In [ ]:
df_train = eliminar_registros_duplicados(df_train)

In [ ]:
df_train.shape

## Normalización

In [ ]:
def escalar_datos(df, scaler_dict=None):
    """
    Escala las características en un DataFrame utilizando MinMaxScaler.

    Parámetros:
    - df: DataFrame: El DataFrame que contiene los datos.
    - scaler_dict: dict: Un diccionario que contiene los objetos scaler utilizados para escalar cada columna (opcional).

    Retorna:
    - df_scaled: DataFrame: El DataFrame con las características escaladas.
    - scaler_dict: dict: Un diccionario que contiene los objetos scaler utilizados para escalar cada columna.
    """
    # Si no se proporciona un diccionario de scalers, inicializar uno
    if scaler_dict is None:
        scaler_dict = {}

    # Iterar sobre cada columna en el DataFrame
    for columna in df.columns:
        # Inicializar un scaler para cada columna si no está en el diccionario
        if columna not in scaler_dict:
            scaler_dict[columna] = MinMaxScaler()
        # Escalar la columna y reemplazar los valores en el DataFrame
        df[columna] = scaler_dict[columna].fit_transform(df[[columna]])

    # Devolver el DataFrame escalado y el diccionario de scalers
    return df, scaler_dict

In [ ]:
df_train, train_scaler_dict = escalar_datos(df_train)

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)